In [44]:
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET

In [47]:
# Create a dataframe from the results of a BehaviorSpace experiment
def create_xml_file(filename, experiment_name, repetitions, run_metrics_every_step, 
                    setup_command, go_command, time_limit_steps, metrics, run_metrics_condition, enumerated_values):
    # Create the root element
    experiments = ET.Element("experiments")
    
    # Create the experiment element
    experiment = ET.SubElement(experiments, "experiment", 
                               name=experiment_name, 
                               repetitions=str(repetitions), 
                               runMetricsEveryStep=str(run_metrics_every_step).lower())
    ET.indent(experiments)
    
    # Add setup and go commands
    ET.SubElement(experiment, "setup").text = setup_command
    ET.indent(experiment)
    ET.SubElement(experiment, "go").text = go_command
    ET.indent(experiment)
    
    # Add time limit
    ET.SubElement(experiment, "timeLimit", steps=str(time_limit_steps))
    ET.indent(experiment)

    # Add metrics
    for var in metrics:
        ET.SubElement(experiment, "metric").text=var
        ET.indent(experiment)    

    # Add run metrics condition
    ET.SubElement(experiment, "runMetricsCondition").text = run_metrics_condition
    ET.indent(experiment)

    # Add enumerated values
    for var, values in enumerated_values.items():
        evs = ET.SubElement(experiment, "enumeratedValueSet", variable=var)
        for value in values:
            ET.SubElement(evs, "value", value=str(value))
            ET.indent(experiment)    
    # Add stepped values
    #for var, params in stepped_values.items():
    #    ET.SubElement(experiment, "steppedValueSet", variable=var, first=str(params['first']), step=str(params['step']), last=str(params['last']))
    #    ET.indent(experiment)
    
    # Create the tree and write to file
    with open(filename, 'wb') as f:
        f.write('<?xml version="1.0" encoding="UTF-8"?>\n'.encode('utf8'))
        f.write('<!DOCTYPE experiments SYSTEM "behaviorspace.dtd">\n'.encode('utf8'))
        ET.ElementTree(experiments).write(f, encoding="UTF-8", xml_declaration=False,)

In [51]:
# Example usage
create_xml_file(filename="example.xml", experiment_name="My Experiment", repetitions=10, run_metrics_every_step=False,
            setup_command="setup", 
            go_command="go", 
            time_limit_steps=10000, 
            metrics=["average-whale-hunt","average-help-others","average-fitness"],
            run_metrics_condition='ticks > 500',
            enumerated_values= {"mutation-prob": [0.1]})

In [33]:
# Split a variable range into a number of splits and sample for LHS
def split_variable_range_and_sample(start,end,splits):
    samples=list()
    for (a,b) in zip(np.linspace(start,end,splits),np.linspace(start,end,splits)[1:]):
        samples.append(np.random.uniform(a,b))
    np.random.shuffle(samples)
    return samples

In [37]:
# Latin Hypercube Sampling
def LHsampling(variables,samples):
    # Create a dataframe with all the combinations of the variables
    df = pd.DataFrame(index=range(samples))
    for var, params in variables.items():
        if params['type'] == 'range':
            df[var] = split_variable_range_and_sample(params['start'],params['end'],samples+1)
        elif params['type'] == 'list':
            df[var] = np.random.choice(params['values'],samples)
    return df

In [39]:
kawesqar_parameters = {'mu-logistic-whaling-success': {'type': 'range', 'start': 0, 'end': 1.5},
                        'whaling-harm-prob': {'type': 'range', 'start': 0, 'end': 1},
                        'theta': {'type': 'range', 'start': 0, 'end': 1},
                        'mutation-prob': {'type': 'range', 'start': 0, 'end': 1}}

In [42]:
lhs=LHsampling(kawesqar_parameters,100)
lhs.head()

,mu-logistic-whaling-success,whaling-harm-prob,theta,mutation-prob
0,0.924209,0.822310,0.765205,0.057368
1,0.303722,0.143558,0.798391,0.489101
2,0.271685,0.162775,0.531135,0.045984
3,0.637534,0.366795,0.890892,0.610431
4,0.097664,0.796538,0.174340,0.693373
